In [2]:
import getpass
import os
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain.chat_models import init_chat_model
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma


os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

In [7]:
model_name = os.getenv("LM_MODEL")
base_url = os.getenv("OPENAI_BASE_URL")

model = init_chat_model(
    model_name,
    model_provider="openai",
    base_url=base_url,
    api_key="lm-studio"
    )

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="./chomksy-db", 
)

/Users/mmoise/Documents/Learning/chomsky-rag-agent/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Load, Split, Embed, Store

In [ ]:
import requests
from bs4 import BeautifulSoup

url = "https://chomsky.info/articles/"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

article_links = []
for a in soup.select(".entry-content a"):
    href = a.get("href", "")
    # Avoid anchors, archives, and index pages
    if href.startswith("https://chomsky.info/") and "/articles/" not in href and "/books/" not in href and "#" not in href and "pdf" not in href and href not in article_links:
        article_links.append(href)

# If the above yields too few, fallback to all single-article permalinks in the list
if len(article_links) < 10:
    article_links = []
    for a in soup.find_all("a", href=True):
        href = a["href"]
        if href.startswith("https://chomsky.info/") and ("articles/" not in href) and ("#" not in href) and ("pdf" not in href) and href not in article_links:
            article_links.append(href)

print(f"Found {len(article_links)} article links")
loader = WebBaseLoader(article_links)

docs = loader.load()
# The main article links are under <div class="entry-content"> in a <ul> (unordered list)
# Each article link is in an <a> tag (with possibly some in blue).
# Let's parse links from that specific list, and then load those links.
from typing import List

def get_chomsky_article_links(soup) -> List[str]:
    # Find the "entry-content" div
    entry = soup.find("div", class_="entry-content")
    article_links = []
    if entry:
        # Find the first <ul> inside entry-content
        ul = entry.find("ul")
        if ul:
            for a in ul.find_all("a", href=True):
                href = a["href"]
                # Exclude PDFs and duplicates
                if href.startswith("https://chomsky.info/") and "pdf" not in href and href not in article_links:
                    article_links.append(href)
    return article_links

article_links = get_chomsky_article_links(soup)
print(f"EXACT, from main <ul>: {len(article_links)} article links found:")
for link in article_links[:5]:
    print(link)

if len(article_links) < 10:
    print("Warning: <ul> found less than 10 articles; falling back to old method.")
    article_links = []
    for a in soup.find_all("a", href=True):
        href = a["href"]
        if href.startswith("https://chomsky.info/") and ("articles/" not in href) and ("#" not in href) and ("pdf" not in href) and href not in article_links:
            article_links.append(href)

print(f"Final: {len(article_links)} article links.")
loader = WebBaseLoader(article_links)
docs = loader.load()
total_chars = sum(len(doc.page_content) for doc in docs)
print(f"Total characters across all pages: {total_chars}")

Found 310 article links
EXACT, from main <ul>: 0 article links found:
Final: 310 article links.


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# chunk_overlap specifies the number of characters of overlap between consecutive text chunks,
# which helps preserve context between splits.
# add_start_index, when set to True, includes the start index of the chunk in the split metadata.
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, 
    chunk_overlap=200,     
    add_start_index=True  
)
all_splits = text_splitter.split_documents(docs)
print(f"Split into {len(all_splits)} chunks")

Split into 11609 chunks


In [25]:
# Add documents in batches to avoid exceeding Chroma's max batch size
batch_size = 5000  
document_ids = []

for i in range(0, len(all_splits), batch_size):
    batch = all_splits[i:i + batch_size]
    batch_ids = vector_store.add_documents(batch)
    document_ids.extend(batch_ids)
    print(f"Added batch {i//batch_size + 1}/{(len(all_splits) + batch_size - 1)//batch_size} ({len(batch)} documents)")

print(f"\nTotal documents added: {len(document_ids)}")

Added batch 1/3 (5000 documents)
Added batch 2/3 (5000 documents)
Added batch 3/3 (1609 documents)

Total documents added: 11609
First 3 document IDs: ['1128b528-de59-45d6-8f1a-2dd0377cb378', '033e8051-50c6-482d-8793-2889f39b2b0a', '31218c5f-8802-4f30-b8f8-62bdf1f895d2']


Construct the agent

In [26]:
from langchain.agents import create_agent
from langchain.tools import tool

@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
    """Retrieve information to help answer a query."""
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

tools = [retrieve_context]
prompt = (
    "You are a helpful assistant that can answer questions about Noam Chomsky's articles   ."
)

agent = create_agent(model, tools, system_prompt=prompt)

Test the agent

In [27]:
query = "What did Chomsky say about the Iraq War?"

for event in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


================================ Human Message =================================

What did Chomsky say about the Iraq War?
================================== Ai Message ==================================
Tool Calls:
  retrieve_context (423814462)
 Call ID: 423814462
  Args:
    query: Iraq War
================================= Tool Message =================================
Name: retrieve_context

Source: {'description': 'The Noam Chomsky Website.', 'start_index': 17631, 'language': 'en', 'source': 'https://chomsky.info/199110__/', 'title': 'Aftermath'}
Content: The crisis began with the Iraqi invasion of Kuwait a year ago. There was some fighting, leaving hundreds killed according to Human Rights groups. That hardly qualifies as war. Rather, in terms of crimes against peace and against humanity, it falls roughly into the category of the Turkish invasion of northern Cyprus, Israel’s invasion of Lebanon in 1978, and the U.S. invasion of Panama. In these terms it falls well short of Israe